In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler

In [ ]:
from PIL import Image
import numpy as np
import os
import cv2

In [ ]:
#create a cropper
def crop_image(img):
    w, h = img.size
    return img.crop((0, 30, w, h-70)) # this seems to be the right amount to crop
#cropped_img.show()

In [ ]:
data = []
labels = []

animals = os.listdir("animals")
for animal in animals:
    
    img = Image.open("animals/" + animal)
    img = img.convert('RGB')
    img = crop_image(img)
    resized_image = img.resize((50, 50))
    data.append(np.array(resized_image))
    labels.append(0)


In [ ]:
false_positives = os.listdir("nothing")
for false in false_positives:
    
    img = Image.open("nothing/" + false)
    img = img.convert('RGB')
    img = crop_image(img)
    resized_image = img.resize((50, 50))
    data.append(np.array(resized_image))
    labels.append(1)


In [ ]:
useless_images = os.listdir("useless")
for useless in useless_images:
    if useless.startswith('.'):
        print(f'{useless} not read')
    else:
        img = Image.open("useless/" + useless)
        img = img.convert('RGB')
        img = crop_image(img)
        resized_image = img.resize((50, 50))
        data.append(np.array(resized_image))
        labels.append(0)

In [ ]:
datagen = ImageDataGenerator(
        rotation_range=10,
        zoom_range = 0.1,
        width_shift_range=0.1,
        height_shift_range=0.1)



In [ ]:
# Converting data and labels to np array
trail_pics = np.array(data)
labels = np.array(labels)

In [ ]:
#shuffling animals and labels to get a good mixture
s = np.arange(trail_pics.shape[0])
np.random.shuffle(s)
trail_pics = trail_pics[s]
labels=labels[s]

In [ ]:
#Make a variable num_classes which is the total 
#number of animal categories and a variable data_length which is size of dataset
num_classes = len(np.unique(labels))
data_length = len(trail_pics)


In [ ]:
#Data split into train and test
(x_train, x_test) = trail_pics[(int)(0.1 * data_length):],trail_pics[:(int)(0.1 * data_length)]
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0
train_length = len(x_train)
test_length = len(x_test)

In [ ]:
#lables split into train and test
(y_train,y_test) = labels[(int)(0.1*data_length):],labels[:(int)(0.1*data_length)]

In [ ]:
datagen.fit(x_train)

In [ ]:
import keras
from keras.utils import np_utils
#One hot encoding
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
print(len(x_train))

In [ ]:

model = Sequential()

model.add(Conv2D(32, kernel_size=5,input_shape=(50, 50, 3), activation = 'relu'))
model.add(Conv2D(32, kernel_size=5, activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size=3,activation = 'relu'))
model.add(Conv2D(64, kernel_size=3,activation = 'relu'))
model.add(MaxPool2D(2,2))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size=3, activation = 'relu'))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(128, activation = "relu"))
model.add(Dropout(0.4))
model.add(Dense(2, activation = "softmax"))

In [ ]:

optimizer=Adam(lr=0.001)
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

model.summary()

In [ ]:
model_try = model.fit_generator(datagen.flow(x_train,y_train, batch_size=64),
                              epochs = 100000, validation_data = (x_test,y_test),
                              verbose = 1, steps_per_epoch=100)

In [ ]:

# ## Saving Model to JSON and weights

# serialize model to JSON
model_json = model.to_json()
with open("UPDATED_model.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("UPDATED_model.h5")
print("Saved model to disk")


In [ ]:
def convert_to_array(img):
    im = cv2.imread(img)
    img = Image.fromarray(im, 'RGB')
    image = img.resize((50, 50))
    return np.array(image)
def get_animal_name(label):
    if label==0:
        return "animal"
    if label==1:
        return "nothing"
    if label==2:
        return "useless"

def predict_animal(file):
    print("Predicting .................................")
    ar = convert_to_array(file)
    ar = ar/255
    label = 1
    a = []
    a.append(ar)
    a = np.array(a)
    score = model.predict(a,verbose = 1)
    print(score)
    label_index = np.argmax(score)
    print(label_index)
    acc = np.max(score)
    animal = get_animal_name(label_index)
    print(animal)
    print("The predicted Animal is a "+ animal +" with accuracy =    "+str(acc))

In [ ]:
from matplotlib.pyplot import imshow
import numpy as np
from PIL import Image

%matplotlib inline
pil_im = Image.open('deer.JPG', 'r')
pil_im = crop_image(pil_im)
imshow(np.asarray(pil_im))
predict_animal("deer.JPG")

In [ ]:
pil_im = Image.open('nothing.JPG', 'r')
pil_im = crop_image(pil_im)
imshow(np.asarray(pil_im))
predict_animal("nothing.JPG")

In [ ]:
pil_im = Image.open('useless.JPG', 'r')
pil_im = crop_image(pil_im)
imshow(np.asarray(pil_im))
predict_animal("useless.JPG")